# Ridge

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

## Get Data

In [2]:
from common.data import get_data
from common.paths import HEALTHY

X, Y, demographics, population = get_data(label_path=HEALTHY)
ages = demographics['Age']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Population: {population}')

X: (106, 34716) | Y: 16 | Age: (106,) | Population: healthy


In [3]:
from common.scoring import unimetric_scorer, multimetric_scorer, N_PERM, SCORING, RKF_10_10
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import cross_validate, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Run for one target, one age bin

In [4]:
selected_target = "WISC_VCI"
y = Y[selected_target]

print(f'y: {y.shape}')

y: (106,)


In [91]:
from common.binning import bin_by_age

bins = bin_by_age(X, y, ages, y)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

Bin 1: (117, 34716) | Bin 2: (156, 34716) | Bin 3: (117, 34716)


### Set up the model pipeline, metrics, and cross-validation approach

In [5]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
age_group = 'all'

estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], 
                                        scoring=unimetric_scorer, cv=RKF_10_10)]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cv = pipe['ridgecv']

print(f'Target: {selected_target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

Target: WISC_VCI | Alpha: 15000 | Score: 0.25
CPU times: user 1min 48s, sys: 1.57 s, total: 1min 49s
Wall time: 11.1 s


In [8]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
bin_label = 'all'

estimators = [StandardScaler(), MultiOutputRegressor(
    RidgeCV(alphas=[a for a in range(5000, 55000, 5000)], scoring=regression_scorer, cv=rkf))]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cvs = pipe['multioutputregressor'].estimators_

for target, ridge_cv in zip(WISC_LEVEL[5], ridge_cvs):
    print(f'Target: {target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

ValueError: y must have at least two dimensions for multi-output regression but has only one.

### Run cross-validation

In [11]:
%%time
estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

scores = cross_validate(pipe, X_cv, y_cv, cv=RKF_10_10, scoring=multimetric_scorer, n_jobs=-1, 
                        return_train_score=True, return_estimator=True)

coefs = np.array([estimator['ridge'].coef_ for estimator in scores['estimator']])
avg_coef = np.mean(coefs, axis=0)
intercepts = np.array([estimator['ridge'].intercept_ for estimator in scores['estimator']])
avg_inte = np.mean(intercepts, axis=0)

print(f'ridge_{population}_{selected_target}_{bin_label}')
for metric in scoring:
    metric_values = scores[metric]
    print(f'Avg {metric}: {np.mean(metric_values):.3f}')

ridge_adhd_WISC_FSIQ_all
Avg train_score: 0.890
Avg test_score: 0.348
CPU times: user 901 ms, sys: 283 ms, total: 1.18 s
Wall time: 6.76 s


In [38]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

coef_fn = f'ridge_{population}_{selected_target}_{bin_label}_coef.npy'
inte_fn = f'ridge_{population}_{selected_target}_{bin_label}_inte.npy'

np.save(join(RIDGE_WEIGHTS, coef_fn), avg_coef)
np.save(join(RIDGE_WEIGHTS, inte_fn), avg_inte)

### ICC

In [34]:
%%time
import pingouin as pg

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.897,873.76,34715,3436884,0.0,"[0.9, 0.9]"
ICC2,Single random raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC3,Single fixed raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC1k,Average raters absolute,0.999,873.76,34715,3436884,0.0,"[1.0, 1.0]"
ICC2k,Average random raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"
ICC3k,Average fixed raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"


### Run permutation test statistic (pts)

In [7]:
%%time
score, _, pvalue = permutation_test_score(
    pipe, X_cv, y_cv, cv=RKF_10_10, scoring=unimetric_scorer, n_permutations=N_PERM, n_jobs=-1)

print(f'ridge_{population}_{selected_target}_{age_group}')
print(f'Alpha: {ridge_cv.alpha_} | Score: {score:.2f} | p-value: {pvalue:.4f}')

ridge_healthy_WISC_VCI_all
Alpha: 15000 | Score: 0.23 | p-value: 0.0319
CPU times: user 51.8 s, sys: 1.27 s, total: 53.1 s
Wall time: 11min 4s


## Run for all targets, all age bins

### Run cross-validation

In [23]:
%%time
from os.path import join
from common.binning import bin_data
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import RIDGE_WEIGHTS, RIDGE_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    X_all, y_all, bin_labels = bin_data(X, y)
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 75000, 5000)], 
                                        scoring=unimetric_scorer, cv=RKF_10_10)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        ridge_cv = pipe['ridgecv']
        
        # Do cross-validation
        estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
        pipe = make_pipeline(*estimators)
        
        scores = cross_validate(pipe, X_cv, y_cv, cv=RKF_10_10, 
                                scoring=unimetric_scorer, n_jobs=-1, 
                                return_train_score=False, 
                                return_estimator=True)
        coefs = np.array([estimator['ridge'].coef_ for estimator in scores['estimator']])
        avg_coef = np.mean(coefs, axis=0)
        intercepts = np.array([estimator['ridge'].intercept_ for estimator in scores['estimator']])
        avg_inte = np.mean(intercepts, axis=0)
        
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': ridge_cv.alpha_,
            'Score': np.mean(scores['test_score']),
        })
        print(results[-1])
        
        coef_fn = f'ridge_{population}_{target}_{bin_label}_coef.npy'
        inte_fn = f'ridge_{population}_{target}_{bin_label}_inte.npy'
        np.save(join(RIDGE_WEIGHTS, coef_fn), avg_coef)
        np.save(join(RIDGE_WEIGHTS, inte_fn), avg_inte)

results_df = pd.DataFrame(results)
display(results_df.round(4))
filename = f'ridge_cv_{population}.csv'
results_df.to_csv(join(RIDGE_RESULTS, filename))

{'Model': 'ridge', 'Population': 'healthy', 'Target': 'WISC_FSIQ', 'Bin': 'All', 'Alpha': 5000, 'Score': 0.057620210944572724}
{'Model': 'ridge', 'Population': 'healthy', 'Target': 'WISC_VSI', 'Bin': 'All', 'Alpha': 5000, 'Score': 0.11931415877840088}
{'Model': 'ridge', 'Population': 'healthy', 'Target': 'WISC_VCI', 'Bin': 'All', 'Alpha': 10000, 'Score': 0.24868165486884972}
{'Model': 'ridge', 'Population': 'healthy', 'Target': 'WISC_FRI', 'Bin': 'All', 'Alpha': 5000, 'Score': -0.10770014104687924}
{'Model': 'ridge', 'Population': 'healthy', 'Target': 'WISC_WMI', 'Bin': 'All', 'Alpha': 15000, 'Score': 0.08821147813461333}
{'Model': 'ridge', 'Population': 'healthy', 'Target': 'WISC_PSI', 'Bin': 'All', 'Alpha': 5000, 'Score': -0.07446769204181103}


,Model,Population,Target,Bin,Alpha,Score
0,ridge,healthy,WISC_FSIQ,All,5000,0.0576
1,ridge,healthy,WISC_VSI,All,5000,0.1193
2,ridge,healthy,WISC_VCI,All,10000,0.2487
3,ridge,healthy,WISC_FRI,All,5000,-0.1077
4,ridge,healthy,WISC_WMI,All,15000,0.0882
5,ridge,healthy,WISC_PSI,All,5000,-0.0745


CPU times: user 15min 45s, sys: 19.4 s, total: 16min 5s
Wall time: 1min 46s


### Run permutation test statistic (pts)

In [ ]:
%%time
from os.path import join
from common.binning import bin_data
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import RIDGE_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    X_all, y_all, bin_labels = bin_data(X, y, ages)
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 75000, 5000)], 
                                        scoring=unimetric_scorer, cv=RKF_10_10)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        ridge_cv = pipe['ridgecv']
        
        # Do permutation test
        estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
        pipe = make_pipeline(*estimators)
        
        score, permutation_scores, pvalue = permutation_test_score(
            pipe, X_cv, y_cv, cv=RKF_10_10, scoring=unimetric_scorer, n_permutations=N_PERM, 
            n_jobs=-1)
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': ridge_cv.alpha_,
            'Score': score,
            'P-value': pvalue,
        })
        print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df.round(4))
filename = f'ridge_pts_{population}.csv'
results_df.to_csv(join(RIDGE_RESULTS, filename))